# Models
Samples of how to create models

In [ ]:
#import required libraries
from azure.ml import MLClient
from azure.ml.entities import Model

To start with lets create a handle to the AML workspace

In [ ]:
#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(subscription_id, resource_group, workspace)

## Create a model from a local file

In [ ]:
file_model = Model(
    local_path="mlflow-model/model.pkl",
    name="local-file-example",
    description="Model created from local file."
)
ml_client.models.create_or_update(file_model)

## Create a model from an mlflow model

In [ ]:
mlflow_model = Model(
    local_path="mlflow-model",
    model_format="mlflow",
    name="local-mlflow-example",
    description="local-mlflow-example"
)
ml_client.models.create_or_update(mlflow_model)